In [1]:
import gymnasium
import ale_py
import argparse
from tensorboardX import SummaryWriter
import cv2
import numpy as np
from einops import rearrange
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import deque
from tqdm import tqdm
import copy
import colorama
import random
import json
import shutil
import pickle
import os
import wandb
import importlib

In [ ]:
import sys
sys.path.append(os.path.abspath("/data/I6347325/work_space/STORM"))# Dynamically reload the modules to reflect any changes

import utils
import sub_models.replay_buffer
import env_wrapper
# import agents
import sub_models.director_agents
import sub_models.functions_losses
import sub_models.world_models
import sub_models.constants
import train

importlib.reload(utils)
importlib.reload(sub_models.replay_buffer)
importlib.reload(env_wrapper)
importlib.reload(sub_models.director_agents)
importlib.reload(sub_models.functions_losses)
importlib.reload(sub_models.world_models)
importlib.reload(sub_models.constants)
importlib.reload(train)

from utils import seed_np_torch, Logger, load_config
from sub_models.replay_buffer import ReplayBuffer
from train import (
    build_single_env,
    build_world_model,
    build_agent,
    joint_train_world_model_agent,
)
from sub_models.constants import DEVICE
print(DEVICE, DEVICE.type)

cuda:1 cuda


/home/I6347325/miniconda3/envs/env_RL/lib/python3.13/site-packages/torchrl/data/replay_buffers/samplers.py:34: UserWarning: Failed to import torchrl C++ binaries. Some modules (eg, prioritized replay buffers) may not work with your installation. This is likely due to a discrepancy between your package version and the PyTorch version. Make sure both are compatible. Usually, torchrl majors follow the pytorch majors within a few days around the release. For instance, TorchRL 0.5 requires PyTorch 2.4.0, and TorchRL 0.6 requires PyTorch 2.5.0.
  warnings.warn(EXTENSION_WARNING)


In [3]:
torch.cuda.set_device(DEVICE)
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [4]:
# ignore warnings
import warnings

warnings.filterwarnings("ignore")
if torch.cuda.is_available():
    torch.cuda.set_device(DEVICE)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

class WandbLogger:
    def __init__(self, run):
        self.run = run

    def log(self, key, value, step=None):
        """Log a key-value pair to wandb with optional step."""
        log_dict = {key: value}
        if step is not None:
            self.run.log(log_dict, step=step)
        else:
            self.run.log(log_dict)

In [5]:
class RunParams:
    def __init__(self, env_name="MsPacman", exp_name = "STORM-Director"):
        self._env_name = env_name
        self.exp_name = exp_name
        self.seed = 1
        self.config_path = "config_files/STORM.yaml"
        self.trajectory_path = f"D_TRAJ/{self._env_name}.pkl"
        self.env_name = f"ALE/{self._env_name}-v5"

        self.conf = load_config(self.config_path)
        self.print_args()
    def print_args(self):
        print(colorama.Fore.GREEN + "Arguments:" + colorama.Style.RESET_ALL)
        print(colorama.Fore.GREEN + "-----------------" + colorama.Style.RESET_ALL)
        print(colorama.Fore.GREEN + "exp_name: " + colorama.Style.RESET_ALL + self.exp_name)
        print(colorama.Fore.GREEN + "seed: " + colorama.Style.RESET_ALL + str(self.seed))
        print(colorama.Fore.GREEN + "config_path: " + colorama.Style.RESET_ALL + self.config_path)
        print(colorama.Fore.GREEN + "trajectory_path: " + colorama.Style.RESET_ALL + self.trajectory_path)
        print(colorama.Fore.GREEN + "env_name: " + colorama.Style.RESET_ALL + self.env_name)
        print(colorama.Fore.GREEN + "-----------------" + colorama.Style.RESET_ALL)
    
run_params = RunParams(env_name="MsPacman", exp_name = "STORM-Director-1")
# set seed
seed_np_torch(seed=run_params.seed)
# tensorboard writer
logger = Logger(path=f"runs/{run_params.exp_name}")
# copy config file
shutil.copy(run_params.config_path, f"runs/{run_params.exp_name}/config.yaml")

Arguments:
-----------------
exp_name: STORM-Director-1
seed: 1
config_path: config_files/STORM.yaml
trajectory_path: D_TRAJ/MsPacman.pkl
env_name: ALE/MsPacman-v5
-----------------


'runs/STORM-Director-1/config.yaml'

In [6]:
print(f"Train Steps: {run_params.conf.JointTrainAgent.SampleMaxSteps}")
print(f"Train Batch Size: {run_params.conf.JointTrainAgent.BatchSize}")
print(f"Train Buffer Max Length: {run_params.conf.JointTrainAgent.BufferMaxLength}")
# Setuop env, models, replay buffer
# getting action_dim with dummy env
dummy_env = build_single_env(
    run_params.env_name, run_params.conf.BasicSettings.ImageSize, seed=0
)
action_dim = dummy_env.action_space.n

# build world model and agent
world_model = build_world_model(run_params.conf, action_dim)
agent = build_agent(run_params.conf, action_dim)
print(f"World model transformer: {world_model.storm_transformer.__class__.__name__}")
# Log the number of parameters for both models
world_model_params = sum(p.numel() for p in world_model.parameters() if p.requires_grad)
agent_params = sum(p.numel() for p in agent.parameters() if p.requires_grad)

# build replay buffer
replay_buffer = ReplayBuffer(
    obs_shape=(run_params.conf.BasicSettings.ImageSize, run_params.conf.BasicSettings.ImageSize, 3),
    num_envs=run_params.conf.JointTrainAgent.NumEnvs,
    max_length=run_params.conf.JointTrainAgent.BufferMaxLength,
    warmup_length=run_params.conf.JointTrainAgent.BufferWarmUp,
    store_on_gpu=run_params.conf.BasicSettings.ReplayBufferOnGPU,
)
# judge whether to load demonstration trajectory
if run_params.conf.JointTrainAgent.UseDemonstration:
    print(
        colorama.Fore.MAGENTA
        + f"loading demonstration trajectory from {run_params.trajectory_path}"
        + colorama.Style.RESET_ALL
    )
    replay_buffer.load_trajectory(path=run_params.trajectory_path)


Train Steps: 25000
Train Batch Size: 128
Train Buffer Max Length: 50000


A.L.E: Arcade Learning Environment (version 0.10.2+c9d4b19)
[Powered by Stella]


World model transformer: StochasticTransformerKVCache


In [7]:
# Initialize wandb
with wandb.init(
    project="Director",  # Replace with your project name
    name=run_params.exp_name,   # Use the experiment name from RunParam
    config = {
        "env_name": run_params.env_name,
        "seed": run_params.seed,
    }
) as run:
    # Log the configuration to wandb
    run.config.update(run_params.conf)
    run.log({"WM_params": f"{world_model_params:.2e}", "Agent_params": f"{agent_params:.2e}"})
    logger = WandbLogger(run)
    # train
    joint_train_world_model_agent(
        env_name=run_params.env_name,
        num_envs=run_params.conf.JointTrainAgent.NumEnvs,
        max_steps=run_params.conf.JointTrainAgent.SampleMaxSteps,
        image_size=run_params.conf.BasicSettings.ImageSize,
        replay_buffer=replay_buffer,
        world_model=world_model,
        agent=agent,
        train_dynamics_every_steps=run_params.conf.JointTrainAgent.TrainDynamicsEverySteps,
        train_agent_every_steps=run_params.conf.JointTrainAgent.TrainAgentEverySteps,
        batch_size=run_params.conf.JointTrainAgent.BatchSize,
        demonstration_batch_size=(
            run_params.conf.JointTrainAgent.DemonstrationBatchSize
            if run_params.conf.JointTrainAgent.UseDemonstration
            else 0
        ),
        batch_length=run_params.conf.JointTrainAgent.BatchLength,
        imagine_batch_size=run_params.conf.JointTrainAgent.ImagineBatchSize,
        imagine_demonstration_batch_size=(
            run_params.conf.JointTrainAgent.ImagineDemonstrationBatchSize
            if run_params.conf.JointTrainAgent.UseDemonstration
            else 0
        ),
        imagine_context_length=run_params.conf.JointTrainAgent.ImagineContextLength,
        imagine_batch_length=run_params.conf.JointTrainAgent.ImagineBatchLength,
        save_every_steps=run_params.conf.JointTrainAgent.SaveEverySteps,
        seed=run_params.seed,
        logger=logger,
        args=run_params,
    )



wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: riju11-mukherjee (rm_ai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/home/I6347325/miniconda3/envs/env_RL/lib/python3.13/site-packages/torchrl/data/replay_buffers/samplers.py:34: UserWarning: Failed to import torchrl C++ binaries. Some modules (eg, prioritized replay buffers) may not work with your installation. This is likely due to a discrepancy between your package version and the PyTorch version. Make sure both are compatible. Usually, torchrl majors follow the pytorch majors within a few days around the release. For instance, TorchRL 0.5 requires PyTorch 2.4.0, and TorchRL 0.6 requires PyTorch 2.5.0.
  warnings.warn(EXTENSION_WARNING)
A.L.E: Arcade Learning Environment (version 0.10.2+c9d4b19)
[Powered by Stella]


Current env: ALE/MsPacman-v5


  0%|          | 0/25000 [00:00<?, ?it/s]

...

Training World Model...


  File "/data/I6347325/work_space/STORM/train.py", line 275, in joint_train_world_model_agent
    wm_train_metrics = train_world_model(
        replay_buffer=replay_buffer,
    ...<4 lines>...
        # logger=logger,
    )
  File "/data/I6347325/work_space/STORM/train.py", line 108, in train_world_model
    metrics = world_model.update(
        buffer_sample["obs"],
    ...<3 lines>...
        logger=None,
    )
  File "/data/I6347325/work_space/STORM/sub_models/world_models.py", line 601, in update
    reward_loss = self.symlog_twohot_loss_func(reward_hat, reward)
  File "/home/I6347325/miniconda3/envs/env_RL/lib/python3.13/site-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^
  File "/home/I6347325/miniconda3/envs/env_RL/lib/python3.13/site-packages/torch/nn/modules/module.py", line 1750, in _call_impl
    return forward_call(*args, **kwargs)
  File "/data/I6347325/work_spac

BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f36b18234d0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f36b1869b50, execution_count=7 error_before_exec=None error_in_exec=[Errno 32] Broken pipe info=<ExecutionInfo object at 7f36b1869a50, raw_cell="# Initialize wandb
with wandb.init(
    project="D.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Blabs03.dke.unimaas.nl/data/I6347325/work_space/STORM/run_exp.ipynb#W6sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe